# Timeseries 1 - Basic Things (~1h)

## Layout:

- (1) - Pandas basics(~5 min)
- (2) - Easy Plots (~10 min)
- (3) - StatsModel - Analysis (~20 min)
- (4) - Facebook Prophet (~20 min)

### Prerequisites:

First, if needed, install and load some packages.

In [ ]:
# ! pip install matplotlib --upgrade
# ! pip install pandas --upgrade
# ! pip install seaborn --upgrade
# ! pip install plotly --upgrade
# ! pip install pystan --upgrade
# ! pip install statsmodels
# ! pip install fbprophet --upgrade

In [ ]:
import numpy as np
import pandas as pd

# A little primer on Pandas:

pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language. It is THE tool to know when performing any kind of "on-disk" data analysis. Basically, there are two main components of the pandas library:

- Series : data with an index
- Dataframes : Multiple series with one index (think spreadsheets)



## Series
The first main data type we will learn about for pandas is the Series data type.

#### A Series is very similar to a NumPy array The differences is that a Series can have axis labels, meaning it can be indexed by a label, instead of just a number location. It also can hold any arbitrary Python Object.

You can create series from many python structures

Here is some data:

In [ ]:
labels = ['a','b','c']  
a_list = [1,2,3]
a_nparr = np.array([1,2,3])
a_dict = {'a':1,'b':2,'c':3}

Here we create three series with the same data (the list `[1,2,3]`) 

In [ ]:
pd.Series(data=a_list)
pd.Series(data=a_list,index=labels)
pd.Series(a_list,labels)

You can also use numpy arrays or dicts. The cool thing about dicts is that they contain both labels and data.

In [ ]:
pd.Series(a_nparr)
pd.Series(a_nparr,labels)
pd.Series(a_dict)

Series can hold anything, even python functions.

In [ ]:
pd.Series([max,min,sum])

here's a serie

In [ ]:
a_serie = pd.Series(data=a_list,index=labels)
a_serie

The values are stored in a np.array accessible with the `.values` property

In [ ]:
a_serie.values

labels are stored accessible with the `.index` property

In [ ]:
a_serie.index

### You may have noticed, but series have Indexes !!

Understanding this is the KEY to pandas series. Pandas uses indexes for fast lookups - "think hashtable"

In [ ]:
home_fruit_inventory = pd.Series([4,2,3,4],index = ['Apple', 'Orange','Cherry', 'Banana'])                                   
needed_fruits = pd.Series([0,1,4,3],index = ['Apple', 'Orange','Cherry', 'Banana'])                                   

Index are usefull for many things, like **not** adding apples and oranges

In [ ]:
home_fruit_inventory - needed_fruits

## DataFrames

DataFrames are pandas main datastructures.  A DataFrame can be considered as an ensemble of Series objects with the same index. It's like an excel spreadsheet within python.

It can be created with some data, an index and columns names

In [ ]:
inventories = [[4,2,3,4],[0,1,3,4],[0,0,3,1],[1,None,2,0]]

df = pd.DataFrame(inventories,index=['Apple', 'Orange','Cherry', 'Banana'],columns=["Home","Store1","Store2","Store3"])
df

## What's inside ?

Pandas offer many built-in functions to have a quick overview of a dataframe's data.

### Dataframe information functions

In [ ]:
df.describe() # Get some stats

In [ ]:
df.dtypes # The data types

In [ ]:
df.info() # Some more info

## Selecting data from a dataframe

The `df[...]` indexing usually work as intended when you're used to numpy. But one must be careful, it can become ambiguous.

In [ ]:
df['Home']  # <-- this returns the Series object "Home"
# or df.Home

`df.loc` works by "index/label" and `df.iloc` works by positionning

In [ ]:
df.loc["Orange"] # or by "position" df.iloc[1]

Avoid the "double" select and use .loc[r,c] or .iloc[r,c] like in numpy "x[row,column]"

In [ ]:
# same as df["Home"]["Apple"] <--- This is not recommended
df.loc["Apple","Home"] 

Of course you can select things easily

In [ ]:
print(df == 0)
df[df["Home"]==0]

And don't forget: it's just numpy under the hood !

In [ ]:
df.values # you get the numpy array (where series are axis=0)

## Recap:

- Pandas main structure are dataframe which are simply concatenated series which share the same index.
- Series are essentially lists, with indexes.

[Their documentation is here](https://pandas.pydata.org/pandas-docs/stable/)



# Pandas and Timeseries

Ok, let's dig in more time series related things

## Data : Bike Sharing Demand data

You are provided hourly rental data spanning two years. 

At first, we only consider two data fields:

- datetime - hourly date + timestamp  
- count - number of total rentals


## Loading data with read_csv:

We do two specific things while loading:

- `usecols`: We only consider the datetime and the count series
- `parse_dates` : We parse the datetime serie as dates

NB: [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html?highlight=read_csv#pandas.read_csv) has a TON of options, be sure to check them

In [ ]:
#lets load the data and only consider the count as a serie.
df = pd.read_csv("https://raw.githubusercontent.com/cedias/csvdata/master/train.csv",parse_dates=["datetime"],usecols=['datetime','count'])

df.head()

Ok, what can we do with this simple raw serie ?

### (Todo) first things first: 
Answer those simple questions:

- How many observations do we have ? (10886)
- What is the min/maximum value (1/977)
- Are there missing values ? (Nope)


In [ ]:
## To complete

## Setting time as the index

For now, the serie is indexed by integers (0,1,2,3,...) which can make it hard to find specific days/hours
It would be easier if we could directly use dates to select observations.

To do so, we can set the datetime as the dataframe index by using the `df.set_index` method

In [ ]:
time_indexed = df.set_index("datetime") #here
time_indexed.head()

In [ ]:
time_indexed.reset_index().head() #reverses the "set_index"

In [ ]:
time_indexed.reset_index(drop=True).head() #reverses the "set_index" but discards the index

### (todo) Select the counts of march/april 2011

**Note**: the range selection here is inclusive $[start:end]$ whereas on arrays it's $[start:end[$

In [ ]:
## To complete

### (TODO) Decomposing dates

One reason of why it's really useful to parse dates (besides use them as index) is because it can be easily used for feature building:

Indeed, it's easy to understand that the bike demand might vary between days (week-days/end) or season (summer/winter). Fortunately, all these informations can be readily extrapolated from datetimes by calling one of the many attribute [datetime-data](https://pandas.pydata.org/pandas-docs/stable/reference/arrays.html#datetime-data) such as `.minute` or `.day`.



In [ ]:
df["minutes"] = df.datetime.dt.minute
df["hour"]  = df.datetime.dt.hour
df["day"]  = # To Complete
df["month"]  = # To Complete
df["year"]  = # To Complete
df["weekday"]  = # To Complete


time_indexed = df
time_indexed = time_indexed.set_index("datetime")
time_indexed.head()


## (b) Easy Plotting

The best way to visualize time series are plots. To make plots in python, there are LOT of existing options, here we'll concentrate on two:

- Matplotlib
- Seaborn

## Matplotlib : The classic one

Matplotlib is integrated in pandas and
[Pandas can automagically plot things using matplotlib](https://pandas.pydata.org/pandas-docs/version/0.23.4/api.html#api-dataframe-plotting). Let's compare quickly the two ways of using matplotlib


### Let's say we want to visualize the bike count on the fifth day:

#### 1 - The RAW way : calling `plt.plot`


In [ ]:
%matplotlib inline             
#Makes sure you get an image in notebook
import matplotlib.pyplot as plt

day_number = 5
day_offset = (day_number-1)*23
plt.plot(time_indexed["count"].values[day_offset:day_offset+23])

# In truth,
# plt.plot(time_indexed.loc["20110105","count"].values) would have worked just fine.

plt.show()                      # Shows plot

#### 2 -  The pandas way
with pandas it's much easier:
(and you get free $x$ labels)

In [ ]:
time_indexed.loc["20110105","count"].plot()
plt.show()

## Plot Types

There are multiple plot types built int:

<pre>
df.plot.hist()     histogram
df.plot.bar()      bar chart
df.plot.barh()     horizontal bar chart
df.plot.line()     line chart
df.plot.area()     area chart
df.plot.scatter()  scatter plot
...
</pre>

NOTE: You can also call specific plots by passing their name as an argument, as with `df.plot(kind='area')`.

## (TODO) What if we want to plot a bunch of days on the same $x$ axis ?

- Plot days 2,4,6,8,12 on the same x axis which index goes from 0 to 23.

In [ ]:
for day in [2,4,6,8,12]:
    day_number = day
    # to complete
    
plt.show()

### (Todo) The following code does not plot days on the same $x$ axis, fix it !

In [ ]:
# This doesn't work well -> Why ?
time_indexed.loc["20110102","count"].plot() # To FIX !!!!
time_indexed.loc["20110104","count"].plot() # To FIX !!!!
time_indexed.loc["20110106","count"].plot() # To FIX !!!!
time_indexed.loc["20110108","count"].plot() # To FIX !!!!
time_indexed.loc["20110112","count"].plot() # To FIX !!!!
plt.show()

## Seaborn

[Seaborn](https://seaborn.pydata.org/index.html) is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics.

Here is some of the functionality that seaborn offers:

    - A dataset-oriented API for examining relationships between multiple variables
    - Specialized support for using categorical variables to show observations or aggregate statistics
    - Options for visualizing univariate or bivariate distributions and for comparing them between subsets of data
    - Automatic estimation and plotting of linear regression models for different kinds dependent variables
    - Convenient views onto the overall structure of complex datasets
    - High-level abstractions for structuring multi-plot grids that let you easily build complex visualizations
    - Concise control over matplotlib figure styling with several built-in themes
    - Tools for choosing color palettes that faithfully reveal patterns in your data

Seaborn aims to make visualization a central part of exploring and understanding data. Its dataset-oriented plotting functions operate on dataframes and arrays containing whole datasets and internally perform the necessary semantic mapping and statistical aggregation to produce informative plots.

#### What's interesting with seaborn is that it's tightly integrated with Pandas: 

Recall our `time_indexed` dataframe

In [ ]:
time_indexed.head()

Let's say we want to see how does the bike rental count evolves through a day.
We can simply say we want to see a [line plot](https://seaborn.pydata.org/generated/seaborn.lineplot.html#seaborn.lineplot) of the count through the hours. Seaborn does all the handywork:

In [ ]:
import seaborn as sns

sns.lineplot(data=time_indexed, x="hour",y="count")

Does it changes through the years ? We can simply add a `hue` on the year variable

In [ ]:
sns.lineplot(data=time_indexed, x="hour",y="count",hue="year")

#### (TODO) Is there a difference between week days and weekend days ? what could we plot to see this ?

In [ ]:
# To Complete

### This was just a glimpse of seaborn

Be sure to have a look [at their documentation](https://seaborn.pydata.org/tutorial.html)

## What if we want to see the bigger picture ?

In [ ]:
time_indexed["06-2011"]

Let's plot the 19 first day of a month (june 2011):

In [ ]:
month_data = time_indexed.loc["06-01-2011":"06-19-2011","count"]

month_data.plot(figsize=(25,12))

# Simple forecasts : can we predict the two future days ?

In [ ]:
month_data_train = time_indexed.loc["06-01-2011":"06-17-2011"].copy()
month_data_test = time_indexed.loc["06-18-2011":"06-19-2011"].copy()

month_data_train["count"].plot(figsize=(25,12))
month_data_test["count"].plot(figsize=(25,12))

In [ ]:
month_data_train

Let's start with a naive hypothesis: "tomorrow will be the same as today". However, instead of a model like $\hat{y}_{t} = y_{t-1}$ (which is actually a great baseline for any time series prediction problems and sometimes is impossible to beat), we will assume that the future value of our variable depends on the average of its $k$ previous values. Therefore, we will use the **moving average**.

$\hat{y}_{t} = \frac{1}{k} \displaystyle\sum^{k}_{n=1} y_{t-n}$

Unfortunately, we cannot make predictions far in the future - in order to get the value for the next step, we need the previous values to be actually observed. But moving average has another use case - smoothing the original time series to identify trends. Pandas has an implementation available with [`DataFrame.rolling(window).mean()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html). The wider the window, the smoother the trend. In the case of very noisy data, which is often encountered in finance, this procedure can help detect common patterns.



In [ ]:
month_data_train["ma_2h"] = month_data_train["count"].rolling(window=2).mean()
month_data_train["ma_6h"] = month_data_train["count"].rolling(window=6).mean()

month_data_train[["count","ma_2h","ma_6h"]].plot(figsize=(25,12)) 



### => as it can be seen on the graph, it's hard to shift the mean forward to predict accurately.

## Exponential smoothing

Now, let's see what happens if, instead of weighting the last $k$ values of the time series, we start weighting all available observations while exponentially decreasing the weights as we move further back in time. There exists a formula for **[exponential smoothing](https://en.wikipedia.org/wiki/Exponential_smoothing)** that will help us with this:

$$\hat{y}_{t} = \alpha \cdot y_t + (1-\alpha) \cdot \hat y_{t-1} $$

Here the model value is a weighted average between the current true value and the previous model values. The $\alpha$ weight is called a smoothing factor. It defines how quickly we will "forget" the last available true observation. The smaller $\alpha$ is, the more influence the previous observations have and the smoother the series is.

Exponentiality is hidden in the recursiveness of the function – we multiply by $(1-\alpha)$ each time, which already contains a multiplication by $(1-\alpha)$ of previous model values

This is also [built in pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html)

In [ ]:

month_data_train["ewm_05"] = month_data_train["count"].ewm(alpha=0.5,adjust=False).mean()
month_data_train["ewm_01"] = month_data_train["count"].ewm(alpha=0.1,adjust=False).mean()

month_data_train[["count","ewm_05","ewm_01"]].plot(figsize=(25,12)) 

# Statsmodels & Prophets -- Straightforward series analysis & Forecasting

## Statsmodels 

### First, you can do the same things as with pandas: like exp. smoothing:

#### (a) Setting a DatetimeIndex Frequency
Note that our DatetimeIndex does not have a frequency. In order to build a Holt-Winters smoothing model, statsmodels needs to know the frequency of the data (whether it's daily, monthly etc.). Since observations occur each hour, we'll use H.<br>A full list of time series offset aliases can be found <a href='http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases'>here</a>.

In [ ]:
month_data.index.freq = "H"
month_data_train.index.freq = "H"
month_data_test.index.freq = "H"
month_data_train.index



Simple exp. smoothing yields the same values:

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

#For some reason, when optimized=False is passed into .fit()
#the statsmodels SimpleExpSmoothing function shifts fitted values down one row.
#We fix this by adding .shift(-1) after .fittedvalues

month_data_train['sm_ewm_05']  = SimpleExpSmoothing(month_data_train["count"]).fit(smoothing_level=0.5,optimized=False).fittedvalues.shift(-1)
month_data_train['sm_ewm_01']  = SimpleExpSmoothing(month_data_train["count"]).fit(smoothing_level=0.1,optimized=False).fittedvalues.shift(-1)

month_data_train[["count","sm_ewm_05","sm_ewm_01"]].plot(figsize=(25,12)) 

month_data_train[["ewm_05","sm_ewm_05","ewm_01","sm_ewm_01"]].head()

## Forecasting with `.forecast`

In [ ]:
model = SimpleExpSmoothing(month_data_train["count"]).fit(smoothing_level=0.5,optimized=False)

month_data_test["ewm_05"] = model.forecast(48) # we forecast on 2 days

month_data_test[["count","ewm_05"]].plot(figsize=(25,12))


### Holt-Winters Methods
In the previous cells  we applied <em>Simple Exponential Smoothing</em> using just one smoothing factor $\alpha$ (alpha). This failed to account for other contributing factors like trend and seasonality.

In this section we'll look at <em>Double</em> and <em>Triple Exponential Smoothing</em> with the <a href='https://otexts.com/fpp2/holt-winters.html'>Holt-Winters Methods</a>. 

In <strong>Double Exponential Smoothing</strong> (aka Holt's Method) we introduce a new smoothing factor $\beta$ (beta) that addresses trend:

\begin{split}l_t &= (1 - \alpha) l_{t-1} + \alpha x_t, & \text{    level}\\
b_t &= (1-\beta)b_{t-1} + \beta(l_t-l_{t-1}) & \text{    trend}\\
y_t &= l_t + b_t & \text{    fitted model}\\
\hat y_{t+h} &= l_t + hb_t & \text{    forecasting model (} h = \text{# periods into the future)}\end{split}

Because we haven't yet considered seasonal fluctuations, the forecasting model is simply a straight sloped line extending from the most recent data point. We'll see an example of this in upcoming lectures.

With <strong>Triple Exponential Smoothing</strong> (aka the Holt-Winters Method) we introduce a smoothing factor $\gamma$ (gamma) that addresses seasonality:

\begin{split}l_t &= (1 - \alpha) l_{t-1} + \alpha x_t, & \text{    level}\\
b_t &= (1-\beta)b_{t-1} + \beta(l_t-l_{t-1}) & \text{    trend}\\
c_t &= (1-\gamma)c_{t-L} + \gamma(x_t-l_{t-1}-b_{t-1}) & \text{    seasonal}\\
y_t &= (l_t + b_t) c_t & \text{    fitted model}\\
\hat y_{t+m} &= (l_t + mb_t)c_{t-L+1+(m-1)modL} & \text{    forecasting model (} m = \text{# periods into the future)}\end{split}

Here $L$ represents the number of divisions per cycle. In our case looking at monthly data that displays a repeating pattern each year, we would use $L=12$.

In general, higher values for $\alpha$, $\beta$ and $\gamma$ (values closer to 1), place more emphasis on recent data.

<h3>Related Functions:</h3>
<tt><strong><a href='https://www.statsmodels.org/stable/generated/statsmodels.tsa.holtwinters.SimpleExpSmoothing.html'>statsmodels.tsa.holtwinters.SimpleExpSmoothing</a></strong><font color=black>(endog)</font>&nbsp;&nbsp;&nbsp;&nbsp;
Simple Exponential Smoothing<br>
<strong><a href='https://www.statsmodels.org/stable/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html'>statsmodels.tsa.holtwinters.ExponentialSmoothing</a></strong><font color=black>(endog)</font>&nbsp;&nbsp;
    Holt-Winters Exponential Smoothing</tt>
    
<h3>For Further Reading:</h3>
<tt>
<strong>
<a href='https://www.itl.nist.gov/div898/handbook/pmc/section4/pmc43.htm'>NIST/SEMATECH e-Handbook of Statistical Methods</a></strong>&nbsp;&nbsp;<font color=black>What is Exponential Smoothing?</font></tt>

___
## Double Exponential Smoothing
Where Simple Exponential Smoothing employs just one smoothing factor $\alpha$ (alpha), Double Exponential Smoothing adds a second smoothing factor $\beta$ (beta) that addresses trends in the data. Like the alpha factor, values for the beta factor fall between zero and one ($0<\beta≤1$). The benefit here is that the model can anticipate future increases or decreases where the level model would only work from recent calculations.

We can also address different types of change (growth/decay) in the trend. If a time series displays a straight-line sloped trend, you would use an <strong>additive</strong> adjustment. If the time series displays an exponential (curved) trend, you would use a <strong>multiplicative</strong> adjustment.

As we move toward forecasting, it's worth noting that both additive and multiplicative adjustments may become exaggerated over time, and require <em>damping</em> that reduces the size of the trend over future periods until it reaches a flat line.

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

#additive
month_data_train['doubleEs_add'] =ExponentialSmoothing(month_data_train["count"], trend='add').fit().fittedvalues.shift(-1)

#multiplicative
month_data_train['doubleEs_mul'] =ExponentialSmoothing(month_data_train["count"], trend='mul').fit().fittedvalues.shift(-1)

month_data_train[["count","doubleEs_add","doubleEs_mul"]].plot(figsize=(25,12)) 


In [ ]:
model2 = ExponentialSmoothing(month_data_train["count"], trend='add').fit()
model2mul = ExponentialSmoothing(month_data_train["count"], trend='mul').fit()

month_data_test["doubleEs_add"] = model2.forecast(48) # we forecast on 2 days
month_data_test["doubleEs_mul"] = model2mul.forecast(48) # we forecast on 2 days

month_data_test[["count","doubleEs_add","doubleEs_mul"]].plot(figsize=(25,12))

## Triple Exponential Smoothing
Triple Exponential Smoothing, the method most closely associated with Holt-Winters, adds support for both trends and seasonality in the data. 


In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

month_data_train['tripleEs_add'] =ExponentialSmoothing(month_data_train["count"], trend='add',seasonal='add',seasonal_periods=24).fit().fittedvalues.shift(-1)

month_data_train[["count","doubleEs_add","doubleEs_mul","tripleEs_add"]].plot(figsize=(25,12)) 


## (TODO) Forecast with triple exponential smoothing

In [ ]:
model3 = ExponentialSmoothing(month_data_train["count"], trend='mul',seasonal='add',seasonal_periods=24).fit()

month_data_test["tripleEs_add"] = # >>TO COMPLETE<< # we forecast on 2 days

month_data_test[["count","tripleEs_add"]].plot(figsize=(25,12))

## There are many models to model time series as "moving averages":

### [Be sure to check statsmodel docs](https://www.statsmodels.org/stable/index.html)

##  Prophet 

> Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.


- **Accurate and fast** : Prophet is used in many applications across Facebook for producing reliable forecasts for planning and goal setting. We’ve found it to perform better than any other approach in the majority of cases. We fit models in Stan so that you get forecasts in just a few seconds.

- **Fully automatic** : Get a reasonable forecast on messy data with no manual effort. Prophet is robust to outliers, missing data, and dramatic changes in your time series.

- **Tunable forecasts** : The Prophet procedure includes many possibilities for users to tweak and adjust forecasts. You can use human-interpretable parameters to improve your forecast by adding your domain knowledge.

- **Available in R or Python** : We’ve implemented the Prophet procedure in R and Python, but they share the same underlying Stan code for fitting. Use whatever language you’re comfortable with to get forecasts.


**You should really read the paper for Prophet! It is relatively straightforward and has a lot of insight on their techniques on how Prophet works internally!**

Link to paper: https://peerj.com/preprints/3190.pdf

In [ ]:
from fbprophet import Prophet

In [ ]:
month_data.head()

### Formatting data for Prophet:

> The input to Prophet is always a dataframe with two columns: ds and y. The ds (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp. The y column must be numeric, and represents the measurement we wish to forecast.

In [ ]:
prophet_data_train = month_data_train["count"].reset_index().rename({'datetime':'ds', 'count':'y'}, axis='columns').copy()
prophet_data_train.head()

In [ ]:
prophet_data_test = month_data_test["count"].reset_index().rename({'datetime':'ds', 'count':'y'}, axis='columns').copy()
prophet_data_test.head()

### Using Prophet

Prophet follows the sklearn model API. We create an instance of the Prophet class and then call its fit and predict methods.

**NOTE: Prophet by default is for daily data. You need to pass a frequency for sub-daily or monthly data. Info: https://facebook.github.io/prophet/docs/non-daily_data.html**

In [ ]:
mod = Prophet()
mod.fit(prophet_data_train)
future = mod.make_future_dataframe(periods=48, freq='H')
forecast = mod.predict(future)

## Let's see how the forecast performed

In [ ]:
predicted_days = forecast[-48:].set_index("ds").sort_index().copy()
predicted_days["ground_truth"] = month_data_test.sort_index()["count"].values

In [ ]:
predicted_days.head()

### (TODO) plot the prediction:

In [ ]:
predicted_days.reset_index(drop=True) # >> TO COMPLETE << # for some reason, the index messes things up

### => With prophet, you can easily see learnt componants.

In [ ]:
mod.plot_components(forecast)

## [If you have time left, have a look at prophet docs !](https://facebook.github.io/prophet/docs/quick_start.html#python-api)

#### => Especially, try to add a componant yourself like the holidays